In [1]:
import os
os.chdir("../")
%pwd

'/Users/snehalmachan/Coding/textsummarizer'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: int
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [14]:
import textsummarizer.constants.constants as constants
from textsummarizer.utils.common import read_yaml, create_directories


In [15]:
class ConfigurationManager:
    def __init__(
            self,
            params_file_path: Path= constants.PARAMS_FILE_PATH,
            config_file_path: Path= constants.CONFIG_FILE_PATH,):
        
        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)

        create_directories([self.config.artifacts_root,])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config= self.config.model_trainer
        params= self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config= ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps,
        )    

        return model_trainer_config

In [16]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [17]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device= "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model= model_pegasus)

        #loading_data
        dataset_samsum_pt= load_from_disk(self.config.data_path)

        trainer_args= TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay= self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy= self.config.evaluation_strategy, 
            eval_steps= self.config.evaluation_strategy, 
            save_steps= self.config.evaluation_strategy, 
            gradient_accumulation_steps=16
        )
        
        trainer= Trainer(
                        model= model_pegasus, 
                        args= trainer_args,
                        tokenizer= tokenizer, 
                        data_collator= seq2seq_data_collator,
                        train_dataset= dataset_samsum_pt['test'],
                        eval_dataset= dataset_samsum_pt["validation"]
                        )

        trainer.train()

        ##save model

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [19]:
try:
    config= ConfigurationManager()
    model_trainer_config= config.get_model_trainer_config()
    model_trainer_config= ModelTrainer(config= model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

    

[2023-11-27 23:23:14,049: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-27 23:23:14,067: INFO: common: yaml file: params.yaml loaded successfully]


[2023-11-27 23:23:14,079: INFO: common: created directory at :artifacts]
[2023-11-27 23:23:14,105: INFO: common: created directory at :artifacts/model_trainer]


pytorch_model.bin:   2%|▏         | 41.9M/2.28G [00:16<14:51, 2.51MB/s]


KeyboardInterrupt: 